# Análisis de datos utilizando PyRoot


###Cargo los paquetes nesesarias

In [2]:
#from IPython.core.interactiveshell import InteractiveShell
import ROOT
import os
import commands
#InteractiveShell.ast_node_interactivity = "all" # Para tener el modo interactivo completo

Welcome to JupyROOT 6.09/03


In [3]:
ROOT.gROOT.Reset()

###Cargo las Librerías nesesarias

In [4]:
#Esta es la ruta a la librería dinámica de Delphes (debe de ser compilada con la misma versión de root del notebook)
#ROOT.gSystem.AddDynamicPath("/scratch/cms/MadGraph/MG5_aMC_v2_5_2/Delphes/")
ROOT.gSystem.AddDynamicPath("/home/camilo/HEPTools/Delphes/delphes6/")
ROOT.gSystem.AddIncludePath("/home/camilo/HEPTools/Delphes/delphes6/external/")

In [5]:
ROOT.gSystem.Load("libDelphes");

Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootTreeReader.h
   requested to autoload type ExRootTreeReader
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootTreeWriter.h
   requested to autoload type ExRootTreeWriter
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootTreeBranch.h
   requested to autoload type ExRootTreeBranch
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootResult.h
   requested to autoload type ExRootResult
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootClassifier.h
   requested to autoload type ExRootClassifier
Error in cling::AutoloadingVisitor::InsertIntoAutoloadingState:
   Missing FileEntry for ExRootAnalysis/ExRootFilter.h
   requested to autoload type ExRootFil

###Construcción del TChain

In [ ]:
#Obtengo el número y nombre de carpetas run_#
tmp = commands.getstatusoutput('ls /scratch/cms/Datos/Signal_2j_mu-nu-Run0/Events')
#Parto la cadena de caracteres por '\n', y asigno las partes a un arreglo
Runs = tmp[1].split("\n") 

In [ ]:
#Cero el objeto TChain
MainChain=ROOT.TChain("Delphes")

In [ ]:
#Lleno el objeto MainChain con las salidas de MadGraph-Pythia_Delphes
for run in range(len(Runs)):
    imput = "../Datos/Signal_2j_mu-nu-Run0/Events/"+Runs[run]+"/tag_1_delphes_events.root"
    MainChain.Add(imput)
    

In [ ]:
#Creo el objeto ExRootTreeReader
treeReader = ROOT.ExRootTreeReader(MainChain)
numberOfEntries = treeReader.GetEntries()


In [ ]:
#Obtengo las ramas que voy a usar
branchJet = treeReader.UseBranch("Jet")
branchMuon = treeReader.UseBranch("Muon")
branchMissingET = treeReader.UseBranch("MissingET")

In [ ]:
# Inicializo los contadores
cut1=0; cut2=0; cut3=0; cut4=0; cut5=0;

In [ ]:
# Loop sobre los eventos
for entry in range(0, numberOfEntries):
    #Se carga la entrada espesífica
    treeReader.ReadEntry(entry)

    # Primer corte, al menos 2 jet
    if branchJet.GetEntries() > 1:
        cut1 = cut1 +1;
        bcount=0;
        for i in range(0, branchJet.GetEntries() ):
            jet = branchJet.At(i)
            #Cuento cuantos jet hay identificados como b quarks
            if jet.BTag :
                bcount = bcount + 1;
            
        # Segundo corte, al menos 2 jet b       
        if bcount > 1:
            cut2 = cut2 +1;
            
            # If event contains at least 2 muons
            if branchMuon.GetEntries() > 1:
                cut3 = cut3 +1;
                # Tomo los dos muones mas energéticos y comparo sus cargas eléctricas
                muon1 = branchMuon.At(0)
                muon2 = branchMuon.At(1)
                
                #Corte 4: Que los muones mas energéticos tengan cargas diferentes
                if muon1.Charge*muon2.Charge < 0:
                    cut4 = cut4 +1;

                    
                    #Corte 5: muones mas energéticos tengan masa invariante diferente de la del Z
                    MassMuon= (muon1.P4()+muon2.P4()).M();
                    if not((MassMuon>80) and (MassMuon<100)):
                        cut5 = cut5 +1;

In [ ]:
print numberOfEntries;
print cut1;
print cut2;
print cut3;
print cut4;
print cut5;
